In [12]:
import math


def haversine_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    # Convert coordinates from degrees to radians
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    # Haversine formula
    a = math.sin(delta_phi / 2.0) ** 2 + \
        math.cos(phi1) * math.cos(phi2) * \
        math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    # Distance in kilometers
    distance = R * c
    return distance



In [13]:
import numpy as np

# Data points from user example
data_points = [
    {'name': 'Point A', 'Longitude': 3.0588, 'Latitude': 36.7538},
    {'name': 'Point B', 'Longitude': 3.062, 'Latitude': 36.75},
    {'name': 'Point C', 'Longitude': 2.944, 'Latitude': 36.766},
    {'name': 'Point D', 'Longitude': 3.1, 'Latitude': 36.76},
    {'name': 'Point E', 'Longitude': 2.889, 'Latitude': 36.712},
    {'name': 'Point F', 'Longitude': 3.11, 'Latitude': 36.765},
    {'name': 'Point G', 'Longitude': 3.15, 'Latitude': 36.755},
    {'name': 'Point H', 'Longitude': 2.95, 'Latitude': 36.775},
    {'name': 'Point I', 'Longitude': 3.07, 'Latitude': 36.78},
    {'name': 'Point J', 'Longitude': 2.97, 'Latitude': 36.79},
    {'name': 'Point K', 'Longitude': 3.05, 'Latitude': 36.74},
    {'name': 'Point L', 'Longitude': 2.93, 'Latitude': 36.72},
    {'name': 'Point M', 'Longitude': 2.91, 'Latitude': 36.71},
    {'name': 'Point N', 'Longitude': 2.88, 'Latitude': 36.70},
    {'name': 'Point O', 'Longitude': 3.16, 'Latitude': 36.77},
    {'name': 'Point P', 'Longitude': 3.19, 'Latitude': 36.76},
    {'name': 'Point Q', 'Longitude': 3.21, 'Latitude': 36.75},
    {'name': 'Point R', 'Longitude': 3.17, 'Latitude': 36.74},
    {'name': 'Point S', 'Longitude': 2.92, 'Latitude': 36.74},
    {'name': 'Point T', 'Longitude': 2.85, 'Latitude': 36.69}
]

start_point = {'name': 'Start', 'Longitude': 2.9, 'Latitude': 36.75}
all_points = [start_point] + data_points

# Build distance (and time) matrix in minutes at 60 km/h
n = len(all_points)
dist_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if i != j:
            d = haversine_distance(all_points[i]['Latitude'], all_points[i]['Longitude'],
                          all_points[j]['Latitude'], all_points[j]['Longitude'])
            dist_matrix[i, j] = (d / 60) * 60  # minutes

dist_matrix


array([[ 0.        , 14.15429936, 14.43344642,  4.3046479 , 17.85256983,
         4.33763041, 18.78239696, 22.28004558,  5.2503509 , 15.50631521,
         7.65889875, 13.4113492 ,  4.27491027,  4.53620003,  5.83849645,
        23.26828637, 25.85987774, 27.61954921, 24.08299027,  2.10048522,
         8.02321571],
       [14.15429936,  0.        ,  0.50972697, 10.31640869,  3.73458301,
        15.8295161 ,  4.72808903,  8.12612603,  9.97432344,  3.0793928 ,
         8.87476373,  1.72320242, 12.07713169, 14.12662097, 17.02096619,
         9.19326052, 11.70857292, 13.47750855, 10.02592793, 12.46177045,
        19.91625318],
       [14.43344642,  0.50972697,  0.        , 10.66164498,  3.56333894,
        15.98585374,  4.58993537,  7.85982428, 10.35708897,  3.41111606,
         9.32389107,  1.54261132, 12.2267469 , 14.25754533, 17.14700338,
         9.00900837, 11.45754657, 13.18611181,  9.68695623, 12.70113183,
        20.03883569],
       [ 4.3046479 , 10.31640869, 10.66164498,  0.        

In [14]:
from itertools import permutations

# Set time limit
time_limit = 120  # minutes
start_index = 0  # Start point is at index 0

# Store best path
best_path = []
best_visited_count = 0

# Precompute all distances from start to other nodes (excluding itself)
indices = list(range(1, n))  # Exclude start point

# Try all permutations, and track the longest valid one under the time limit
for r in range(n-1, 0, -1):  # Try to visit as many as possible
    for perm in permutations(indices, r):
        current_time = 0
        current_pos = start_index
        valid_path = True
        for next_pos in perm:
            travel_time = dist_matrix[current_pos, next_pos]
            if current_time + travel_time > time_limit:
                valid_path = False
                break
            current_time += travel_time
            current_pos = next_pos
        if valid_path and r > best_visited_count:
            best_path = [start_index] + list(perm)
            best_visited_count = r
    if best_path:
        break  # Found longest valid path

# Convert indices back to point names
optimal_path_names = [all_points[i]['name'] for i in best_path]
optimal_path_names


KeyboardInterrupt: 

In [17]:
from geopy.distance import geodesic
from copy import deepcopy

def estimate_time(p1, p2, speed_kmph=60):
    distance_km = geodesic((p1['Latitude'], p1['Longitude']), (p2['Latitude'], p2['Longitude'])).km
    return (distance_km / speed_kmph) * 60  # minutes

class RouteOptimizerBB:
    def __init__(self, points, start_point, speed=60, time_limit=480):
        self.points = points
        self.start = start_point
        self.speed = speed
        self.time_limit = time_limit
        self.best_path = []
        self.best_count = 0

    def solve(self):
        visited = set()
        self._branch_and_bound(
            current=self.start,
            remaining=self.points,
            path=[self.start],
            time_spent=0,
            visited_set=visited
        )
        return self.best_path[1:], self._calculate_total_time(self.best_path)

    def _calculate_total_time(self, path):
        total = 0
        for i in range(len(path)-1):
            total += estimate_time(path[i], path[i+1], self.speed)
        return total

    def _branch_and_bound(self, current, remaining, path, time_spent, visited_set):
        if time_spent > self.time_limit:
            return

        if len(path) - 1 > self.best_count:
            self.best_path = deepcopy(path)
            self.best_count = len(path) - 1

        for i, point in enumerate(remaining):
            if point['name'] in visited_set:
                continue

            travel_time = estimate_time(current, point, self.speed)
            new_time = time_spent + travel_time

            if new_time <= self.time_limit:
                visited_set.add(point['name'])
                self._branch_and_bound(
                    current=point,
                    remaining=remaining[:i] + remaining[i+1:],
                    path=path + [point],
                    time_spent=new_time,
                    visited_set=visited_set
                )
                visited_set.remove(point['name'])


In [3]:
import random

# Define the bounds for the coordinates
latitude_bounds = (36.69, 36.80)
longitude_bounds = (2.85, 3.21)

# Function to generate random points
def generate_random_points(num_points):
    points = []
    for i in range(num_points):
        latitude = round(random.uniform(latitude_bounds[0], latitude_bounds[1]), 4)
        longitude = round(random.uniform(longitude_bounds[0], longitude_bounds[1]), 4)
        points.append({
            'name': f'Point {chr(65 + i)}',  # Letters A, B, C, ...
            'Longitude': longitude,
            'Latitude': latitude
        })
    return points

# Generate 40 random points
data_points = generate_random_points(40)
data_points


[{'name': 'Point A', 'Longitude': 3.122, 'Latitude': 36.6902},
 {'name': 'Point B', 'Longitude': 2.9917, 'Latitude': 36.6996},
 {'name': 'Point C', 'Longitude': 2.8939, 'Latitude': 36.7582},
 {'name': 'Point D', 'Longitude': 3.0998, 'Latitude': 36.7292},
 {'name': 'Point E', 'Longitude': 3.0992, 'Latitude': 36.7022},
 {'name': 'Point F', 'Longitude': 3.0502, 'Latitude': 36.7737},
 {'name': 'Point G', 'Longitude': 3.0309, 'Latitude': 36.735},
 {'name': 'Point H', 'Longitude': 2.9736, 'Latitude': 36.7482},
 {'name': 'Point I', 'Longitude': 2.9319, 'Latitude': 36.7563},
 {'name': 'Point J', 'Longitude': 3.1883, 'Latitude': 36.7037},
 {'name': 'Point K', 'Longitude': 3.0973, 'Latitude': 36.7867},
 {'name': 'Point L', 'Longitude': 2.9528, 'Latitude': 36.7701},
 {'name': 'Point M', 'Longitude': 3.1712, 'Latitude': 36.7648},
 {'name': 'Point N', 'Longitude': 2.9494, 'Latitude': 36.7466},
 {'name': 'Point O', 'Longitude': 3.1504, 'Latitude': 36.7287},
 {'name': 'Point P', 'Longitude': 2.9254, 

In [8]:
import random
import math
import numpy as np

# Helper: Haversine formula to compute distance between two lat/lon points
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # Earth radius in km
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = (math.sin(dlat / 2) ** 2 +
         math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2)
    return R * 2 * math.asin(math.sqrt(a))

# Setup
speed_kmph = 60
time_limit_minutes = 400


start_point = {'name': 'Start', 'Longitude': 2.9, 'Latitude': 36.75}

# Create a list of all points with the start at index 0
points = [start_point] + data_points
N = len(points)

# Compute travel time matrix in minutes
travel_time = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        if i != j:
            dist = haversine(points[i]['Longitude'], points[i]['Latitude'],
                             points[j]['Longitude'], points[j]['Latitude'])
            time = (dist / speed_kmph) * 60  # in minutes
            travel_time[i][j] = time

# Helper: Calculate total travel time of a given path
def calculate_total_time(path, travel_time):
    total_time = 0
    for i in range(len(path) - 1):
        total_time += travel_time[path[i]][path[i + 1]]
    return total_time

# Simulated Annealing Function
def simulated_annealing(travel_time, initial_temperature, cooling_rate, time_limit):
    # Initialize with a random path (excluding start)
    n = len(travel_time)
    current_solution = list(range(1, n))  # Exclude the start point (index 0)
    random.shuffle(current_solution)
    current_solution = [0] + current_solution  # Add start point at the beginning
    
    current_cost = calculate_total_time(current_solution, travel_time)
    
    temperature = initial_temperature
    
    # Store the best solution found
    best_solution = list(current_solution)
    best_cost = current_cost
    best_visited_points = len(current_solution)  # Number of points visited
    
    while temperature > 1:
        # Create a neighboring solution by swapping two points
        new_solution = list(current_solution)
        i, j = random.sample(range(1, n), 2)  # Randomly pick two points to swap
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
        
        # Calculate new cost
        new_cost = calculate_total_time(new_solution, travel_time)
        
        # Only accept the new solution if it's better or with a certain probability
        if new_cost <= time_limit:
            # Track the number of points visited
            visited_points = len(set(new_solution))  # Count unique points visited
            
            if visited_points > best_visited_points:
                best_solution = list(new_solution)
                best_cost = new_cost
                best_visited_points = visited_points
            
        elif random.uniform(0, 1) < math.exp((current_cost - new_cost) / temperature):
            current_solution = list(new_solution)
            current_cost = new_cost
        
        # Cooling down
        temperature *= cooling_rate
    
    return best_solution, best_cost, best_visited_points

# Setup parameters
initial_temperature = 1000
cooling_rate = 0.99
time_limit = 480  # Minutes

# Example usage
best_solution, best_cost, best_visited_points = simulated_annealing(travel_time, initial_temperature, cooling_rate, time_limit)
print("Best Path:", best_solution)
print("Total Time:", best_cost)
print("Number of Points Visited:", best_visited_points-1)


Best Path: [0, 20, 25, 19, 18, 24, 2, 37, 12, 9, 34, 30, 16, 15, 21, 11, 36, 31, 13, 7, 17, 23, 28, 39, 6, 35, 5, 27, 1, 26, 4, 14, 32, 22, 40, 38, 3, 8, 10, 33, 29]
Total Time: 490.88882444523796
Number of Points Visited: 40
